## PP-ANN Vs. ANNOY

In [34]:
import numpy as np
import time
import os
import urllib.request
import zipfile
import gc

from annoy import AnnoyIndex
from sklearn.preprocessing import StandardScaler
from skpp import ProjectionPursuitRegressor
from sklearn.neighbors import NearestNeighbors

In [35]:
class ProjectionPursuitANN:
    def __init__(self, r=3, stage_maxiter=50, backfit_maxiter=5, tol=1e-4):
        self.r = r
        self.stage_maxiter = stage_maxiter
        self.backfit_maxiter = backfit_maxiter
        self.tol = tol
        self.ppr = None
        self.knn = None
        self.scaler = StandardScaler()
        
    def fit(self, X, k=10):
        try:
            # Scale the data
            X_scaled = self.scaler.fit_transform(X)
            
            # Use a small number of random reference points
            n_samples = X_scaled.shape[0]
            max_targets = min(5, n_samples // 20)
            reference_indices = np.random.choice(n_samples, max_targets, replace=False)
            target = np.zeros((n_samples, max_targets))
            for i, ref_idx in enumerate(reference_indices):
                target[:, i] = np.linalg.norm(X_scaled - X_scaled[ref_idx].reshape(1, -1), axis=1)
            
            # Fit projection pursuit regressor with conservative parameters
            self.ppr = ProjectionPursuitRegressor(
                r=self.r,
                stage_maxiter=self.stage_maxiter,
                backfit_maxiter=self.backfit_maxiter,
                eps_stage=self.tol,
                opt_level='medium',
                random_state=42
            )
            self.ppr.fit(X_scaled, target)
            
            # Project the data
            X_projected = self.ppr.transform(X_scaled)
            
            # Build kNN model on projected data
            self.knn = NearestNeighbors(n_neighbors=k, algorithm='brute')
            self.knn.fit(X_projected)
            
            return self
        except Exception as e:
            print(f"Error fitting PP-ANN: {e}")
            # Fallback to standard kNN
            self.knn = NearestNeighbors(n_neighbors=k)
            self.knn.fit(X)
            return self
    
    def kneighbors(self, X, n_neighbors=None, return_distance=True):
        try:
            if self.ppr is not None:
                X_scaled = self.scaler.transform(X)
                X_projected = self.ppr.transform(X_scaled)
                return self.knn.kneighbors(X_projected, n_neighbors, return_distance)
            else:
                return self.knn.kneighbors(X, n_neighbors, return_distance)
        except Exception as e:
            print(f"Error in kneighbors: {e}")
            indices = np.random.randint(0, self.knn._fit_X.shape[0], 
                                          size=(X.shape[0], n_neighbors or self.knn.n_neighbors))
            if return_distance:
                distances = np.ones_like(indices, dtype=float)
                return distances, indices
            return indices

# ANNOY wrapper for a consistent API
class ANNOYWrapper:
    def __init__(self, n_trees=10, metric='angular'):
        self.n_trees = n_trees
        self.metric = metric
        self.index = None
        self.dim = None
        self.data = None
        
    def fit(self, X, k=10):
        self.data = X
        self.dim = X.shape[1]
        
        self.index = AnnoyIndex(self.dim, self.metric)
        for i, x in enumerate(X):
            self.index.add_item(i, x)
            
        self.index.build(self.n_trees)
        return self
    
    def kneighbors(self, X, n_neighbors=5, return_distance=True):
        indices = []
        distances = []
        for x in X:
            idx, dist = self.index.get_nns_by_vector(x, n_neighbors, include_distances=True)
            indices.append(idx)
            distances.append(dist)
        if return_distance:
            return np.array(distances), np.array(indices)
        else:
            return np.array(indices)

# Load GloVe word embeddings dataset
def load_glove_embeddings(file_path='glove.6B.50d.txt', limit=10000):
    print(f"Loading GloVe embeddings from {file_path}...")
    if not os.path.exists(file_path):
        url = "https://nlp.stanford.edu/data/glove.6B.zip"
        print(f"Downloading GloVe embeddings from {url}...")
        urllib.request.urlretrieve(url, "glove.6B.zip")
        with zipfile.ZipFile("glove.6B.zip", "r") as zip_ref:
            zip_ref.extractall(".")
        print("Download complete")
    
    word_to_index = {}
    embeddings = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= limit:
                break
            values = line.rstrip().split(' ')
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            word_to_index[word] = i
            embeddings.append(vector)
    return np.array(embeddings), word_to_index

In [36]:
def evaluate_similarity_search(embeddings, word_to_index, index_to_word, method, n_queries=10, k=5):
    semantic_pairs = [
        ('king', 'queen'),
        ('man', 'woman'),
        ('france', 'paris'),
        ('japan', 'tokyo'),
        ('computer', 'laptop'),
        ('car', 'vehicle'),
        ('happy', 'sad'),
        ('river', 'lake'),
        ('sun', 'moon'),
        ('doctor', 'hospital')
    ]
    
    results = {}
    valid_pairs = [(w1, w2) for w1, w2 in semantic_pairs 
                   if w1 in word_to_index and w2 in word_to_index][:n_queries]
    
    if not valid_pairs:
        print("No valid semantic pairs found in the vocabulary!")
        return {}
    
    for w1, w2 in valid_pairs:
        idx1 = word_to_index[w1]
        query_vector = embeddings[idx1].reshape(1, -1)
        
        if method.__class__.__name__ == 'ANNOYWrapper':
            _, neighbor_indices = method.kneighbors(query_vector, n_neighbors=k+1)
            neighbor_indices = neighbor_indices[0]
        else:
            _, neighbor_indices = method.kneighbors(query_vector, n_neighbors=k+1)
            neighbor_indices = neighbor_indices[0]
        
        neighbors = [index_to_word[idx] for idx in neighbor_indices if idx != idx1]
        rank = -1
        if w2 in [index_to_word[idx] for idx in neighbor_indices]:
            for i, idx in enumerate(neighbor_indices):
                if index_to_word[idx] == w2:
                    rank = i
                    break
        
        neighbor_words = [index_to_word[idx] for idx in neighbor_indices[:k]]
        results[w1] = {
            'query': w1,
            'expected': w2,
            'neighbors': neighbor_words,
            'found': w2 in neighbor_words,
            'rank': rank
        }
    
    return results

# Benchmark function with adjustable query count (without memory measurements)
def benchmark_ann_methods(embeddings, word_to_index, random_seed=42, n_queries=1000):
    np.random.seed(random_seed)
    index_to_word = {idx: word for word, idx in word_to_index.items()}
    
    n_samples = embeddings.shape[0]
    X_train = embeddings
    k = 10  # Number of neighbors
    
    results = {}
    semantic_results = {}
    
    methods = {
        'PP-ANN (r=15)': ProjectionPursuitANN(r=15, stage_maxiter=500, backfit_maxiter=5),
        'PP-ANN (r=30)': ProjectionPursuitANN(r=30, stage_maxiter=500, backfit_maxiter=5),
        'ANNOY (10 trees)': ANNOYWrapper(n_trees=10, metric='angular'),
        'ANNOY (50 trees)': ANNOYWrapper(n_trees=50, metric='angular'),
        'ANNOY (100 trees)': ANNOYWrapper(n_trees=100, metric='angular'),
        'Exact NN': NearestNeighbors(n_neighbors=k, algorithm='brute')
    }
    
    query_indices = np.random.choice(n_samples, n_queries, replace=False)
    X_query = X_train[query_indices]
    
    print("Building indices for all methods...")
    for name, method in methods.items():
        print(f"  Building index for {name}...")
        method.fit(X_train, k)
    
    for name, method in methods.items():
        print(f"\nBenchmarking query performance for {name}...")
        try:
            # Measure fit time
            t0 = time.time()
            method.fit(X_train, k)
            fit_time = time.time() - t0
            
            # Measure query performance
            chunk_size = 200
            all_distances = []
            all_indices = []
            query_start = time.time()
            for i in range(0, n_queries, chunk_size):
                end_idx = min(i + chunk_size, n_queries)
                chunk = X_query[i:end_idx]
                distances, indices = method.kneighbors(chunk, n_neighbors=k)
                all_distances.append(distances)
                all_indices.append(indices)
            query_time = time.time() - query_start
            distances = np.vstack(all_distances)
            indices = np.vstack(all_indices)
            avg_query_time = query_time / n_queries
            
            # Compute recall for non-exact methods
            if name != 'Exact NN':
                nn = NearestNeighbors(n_neighbors=k, algorithm='brute')
                nn.fit(X_train)
                recall_sum = 0
                for i in range(0, n_queries, chunk_size):
                    end_idx = min(i + chunk_size, n_queries)
                    chunk = X_query[i:end_idx]
                    _, exact_indices = nn.kneighbors(chunk, n_neighbors=k)
                    for j in range(end_idx - i):
                        method_neighbors = set(indices[i+j])
                        exact_neighbors = set(exact_indices[j])
                        recall_sum += len(method_neighbors.intersection(exact_neighbors)) / k
                recall = recall_sum / n_queries
            else:
                recall = 1.0
            
            semantic_eval = evaluate_similarity_search(embeddings, word_to_index, index_to_word, method, n_queries=15)
            semantic_accuracy = sum(result['found'] for result in semantic_eval.values()) / len(semantic_eval)
            
            semantic_results[name] = semantic_eval
            results[name] = {
                'fit_time': fit_time,
                'query_time': avg_query_time,
                'recall@k': recall,
                'semantic_accuracy': semantic_accuracy
            }
        except Exception as e:
            print(f"Error benchmarking {name}: {e}")
            results[name] = {
                'fit_time': float('nan'),
                'query_time': float('nan'),
                'recall@k': float('nan'),
                'semantic_accuracy': float('nan')
            }
    
    return results, semantic_results

# Run the benchmark
if __name__ == "__main__":
    embeddings, word_to_index = load_glove_embeddings(file_path='glove.6B.50d.txt', limit=10000)
    print(f"Loaded {len(embeddings)} word embeddings with {embeddings.shape[1]} dimensions")
    
    results, semantic_results = benchmark_ann_methods(embeddings, word_to_index, n_queries=1000)
    
    print("\nBenchmark Results:")
    print("-" * 100)
    header = f"{'Method':<20} {'Fit Time (s)':<12} {'Query Time (ms)':<16} {'Recall@10':<10} {'Semantic Acc':<12}"
    print(header)
    print("-" * 100)
    
    for name, metrics in results.items():
        query_time_ms = metrics['query_time'] * 1000  # Convert to ms
        print(f"{name:<20} {metrics['fit_time']:<12.2f} {query_time_ms:<16.2f} {metrics['recall@k']:<10.4f} {metrics['semantic_accuracy']:<12.4f}")
    
    print("\nSemantic Search Examples:")
    print("=" * 100)
    for method_name, method_results in semantic_results.items():
        print(f"\n{method_name}:")
        print("-" * 50)
        for query, result in list(method_results.items())[:3]:
            neighbors_str = ", ".join(result['neighbors'])
            found_str = "✓" if result['found'] else "✗"
            print(f"Query: '{query}', Expected: '{result['expected']}', Found: {found_str}")
            print(f"Neighbors: {neighbors_str}\n")
    
    avg_recall = sum(m['recall@k'] for m in results.values()) / len(results)
    avg_semantic = sum(m['semantic_accuracy'] for m in results.values()) / len(results)
    print("\nOverall Benchmark Summary:")
    print(f"Average Recall@10 across all methods: {avg_recall:.4f}")
    print(f"Average Semantic Accuracy across all methods: {avg_semantic:.4f}")
    print(f"Total embeddings benchmarked: {len(embeddings)}")

Loading GloVe embeddings from glove.6B.50d.txt...
Loaded 10000 word embeddings with 50 dimensions
Building indices for all methods...
  Building index for PP-ANN (r=15)...
  Building index for PP-ANN (r=30)...
  Building index for ANNOY (10 trees)...
  Building index for ANNOY (50 trees)...
  Building index for ANNOY (100 trees)...
  Building index for Exact NN...

Benchmarking query performance for PP-ANN (r=15)...

Benchmarking query performance for PP-ANN (r=30)...

Benchmarking query performance for ANNOY (10 trees)...

Benchmarking query performance for ANNOY (50 trees)...

Benchmarking query performance for ANNOY (100 trees)...

Benchmarking query performance for Exact NN...

Benchmark Results:
----------------------------------------------------------------------------------------------------
Method               Fit Time (s) Query Time (ms)  Recall@10  Semantic Acc
----------------------------------------------------------------------------------------------------
PP-ANN (r=15)